<a href="https://colab.research.google.com/github/jennahgosciak/nyc_fire_risk/blob/main/00_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# setup
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import requests
import calendar
import geopandas as gpd
import os.path as os
import scipy.stats
import seaborn.palettes
import seaborn.utils
import sys
from census import Census
from us import states
import http.client, urllib.request, urllib.parse, urllib.error, base64
import config

root= r"C:/Users/Jennah/Desktop/Code/machine-learning-final"
inp= os.join(root, "data", "1_raw")
out= os.join(root, "data", "2_intermediate")

# Estimating Fires in NYC

## Load data on fire dispatch events
* Only structural fires are included


In [2]:
# load data on all fire dispate events for structural fires
url_fire_ev = 'https://data.cityofnewyork.us/resource/8m42-w767.csv?$limit=1000000&$where=INCIDENT_CLASSIFICATION_GROUP="Structural%20Fires"'
fire_ev = pd.read_csv(url_fire_ev)
print(fire_ev.shape)

# if false, then we might not have downloaded all data
print(fire_ev.shape[0])
assert fire_ev.shape[0] < 1000000

(456376, 29)
456376


In [3]:
# create month, date, and year variables
fire_ev["incident_date"]= pd.to_datetime(fire_ev["incident_datetime"]).dt.date
fire_ev["incident_month"]= pd.to_datetime(fire_ev["incident_datetime"]).dt.month
fire_ev["incident_day"]= pd.to_datetime(fire_ev["incident_datetime"]).dt.day
fire_ev["incident_year"]= pd.to_datetime(fire_ev["incident_datetime"]).dt.year

fire_ev["incident_md"]= fire_ev[["incident_month", "incident_day"]].astype(str).apply('-'.join, 1)
print("\nFire events by year")
fire_ev["incident_year"].value_counts().sort_index()


Fire events by year


2005    31247
2006    30546
2007    30651
2008    29163
2009    28753
2010    28961
2011    27595
2012    27908
2013    25097
2014    26531
2015    27403
2016    27442
2017    28259
2018    27119
2019    26154
2020    25035
2021     8512
Name: incident_year, dtype: int64

In [4]:
fire_ev.head()

,starfire_incident_id,incident_datetime,alarm_box_borough,alarm_box_number,alarm_box_location,incident_borough,zipcode,policeprecinct,citycouncildistrict,communitydistrict,...,incident_response_seconds_qy,incident_travel_tm_seconds_qy,engines_assigned_quantity,ladders_assigned_quantity,other_units_assigned_quantity,incident_date,incident_month,incident_day,incident_year,incident_md
0,5.001924e+14,2005-01-01T00:07:32.000,QUEENS,9237,N/SVC RD H. HARDING EXPY & 99 ST,QUEENS,11368.0,110.0,21.0,404.0,...,338,236,3,2,2,2005-01-01,1,1,2005,1-1
1,5.001149e+14,2005-01-01T00:14:40.000,MANHATTAN,1493,BWAY & W125 ST\M.L.KING JR BLVD,MANHATTAN,10027.0,26.0,7.0,109.0,...,266,217,2,2,1,2005-01-01,1,1,2005,1-1
2,5.001088e+14,2005-01-01T00:16:29.000,MANHATTAN,878,2 AVE & 53 ST,MANHATTAN,10022.0,17.0,4.0,106.0,...,298,220,5,4,6,2005-01-01,1,1,2005,1-1
3,5.001088e+14,2005-01-01T00:16:29.000,MANHATTAN,878,2 AVE & 53 ST,MANHATTAN,10022.0,17.0,4.0,106.0,...,298,220,5,4,6,2005-01-01,1,1,2005,1-1
4,5.001065e+14,2005-01-01T00:24:58.000,BROOKLYN,653,LAFAYETTE & CLASSON AVES,BROOKLYN,11238.0,79.0,35.0,303.0,...,226,189,3,2,1,2005-01-01,1,1,2005,1-1


In [5]:
## save file in output folder
fire_ev.to_csv(os.join(out, "fire_dispatch.csv"))

In [57]:
## load in service alarm boxes
alarm_box= gpd.read_file(os.join(inp, "In-Service Alarm Box Locations.geojson"))

## load census tracts
tracts= gpd.read_file("https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/NYC_Census_Tracts_for_2010_US_Census/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=pgeojson")

,OBJECTID,CTLabel,BoroCode,BoroName,CT2010,BoroCT2010,CDEligibil,NTACode,NTAName,PUMA,Shape__Area,Shape__Length,geometry
0,1,9,5,Staten Island,000900,5000900,E,SI22,West New Brighton-New Brighton-St. George,3903,2.497010e+06,7729.016794,"POLYGON ((-74.07921 40.64344, -74.07914 40.643..."
1,2,98,1,Manhattan,009800,1009800,I,MN19,Turtle Bay-East Midtown,3808,1.906016e+06,5534.200308,"POLYGON ((-73.96433 40.75639, -73.96479 40.755..."
2,3,102,1,Manhattan,010200,1010200,I,MN17,Midtown-Midtown South,3807,1.860993e+06,5687.802439,"POLYGON ((-73.97124 40.76094, -73.97170 40.760..."
3,4,104,1,Manhattan,010400,1010400,I,MN17,Midtown-Midtown South,3807,1.864600e+06,5693.036367,"POLYGON ((-73.97446 40.76230, -73.97492 40.761..."
4,5,113,1,Manhattan,011300,1011300,I,MN17,Midtown-Midtown South,3807,1.890907e+06,5699.860640,"POLYGON ((-73.98412 40.75485, -73.98460 40.754..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2160,2161,183.02,2,Bronx,018302,2018302,I,BX14,East Concourse-Concourse Village,3708,1.208595e+06,4627.750710,"POLYGON ((-73.91811 40.83001, -73.91729 40.829..."
2161,2162,196,1,Manhattan,019600,1019600,I,MN34,East Harlem North,3804,1.902453e+06,5776.739392,"POLYGON ((-73.93631 40.80303, -73.93677 40.802..."
2162,2163,242,1,Manhattan,024200,1024200,E,MN34,East Harlem North,3804,3.726642e+06,9379.633899,"POLYGON ((-73.92998 40.80299, -73.93015 40.802..."
2163,2164,69,2,Bronx,006900,2006900,E,BX34,Melrose South-Mott Haven North,3710,2.029126e+06,6537.689395,"POLYGON ((-73.91182 40.82204, -73.91217 40.821..."


In [81]:
# spatial join to get info
alarm_box_t= gpd.sjoin(alarm_box, tracts, how = "left", op = "intersects")
alarm_box_t.head()

,location,latitude,zip,borobox,communitydistict,longitude,box_type,citycouncil,borough,geometry,...,BoroCode,BoroName,CT2010,BoroCT2010,CDEligibil,NTACode,NTAName,PUMA,Shape__Area,Shape__Length
0,3 AVE & 65 ST,40.63932033,11220,B2653,BK07,-74.02354939,ERS,38,Brooklyn,POINT (-74.02355 40.63932),...,3,Brooklyn,007000,3007000,E,BK31,Bay Ridge,4013,1.609298e+06,7191.182362
1,WOODSIDE AVE & 69 ST,40.7426855,11377,Q7917,QN02,-73.89565167,BARS,26,Queens,POINT (-73.89565 40.74269),...,4,Queens,048300,4048300,E,QN50,Elmhurst-Maspeth,4109,2.399981e+06,7509.356297
2,MYRTLE AVE & PALMETTO ST,40.69953211,11237,B0801,QN05,-73.9110349,ERS,34,Brooklyn,POINT (-73.91103 40.69953),...,3,Brooklyn,043900,3043900,E,BK77,Bushwick North,4002,2.152393e+06,8162.503636
3,NEW YORK AVE & LEFFERTS AVE,40.66253364,11225,B1046,BK09,-73.94791393,ERS,40,Brooklyn,POINT (-73.94791 40.66253),...,3,Brooklyn,080600,3080600,E,BK60,Prospect Lefferts Gardens-Wingate,4011,1.790169e+06,6017.644684
4,RIVER & NORTH 3 STS,40.71837562,11211,B0109,BK01,-73.96462115,ERS,33,Brooklyn,POINT (-73.96462 40.71838),...,3,Brooklyn,055500,3055500,I,BK73,North Side-South Side,4001,2.838296e+06,7349.098694


In [77]:
fire_ev["alarm_box_number_char"]= fire_ev["alarm_box_number"].astype(str).str.pad(width = 4, fillchar = "0")

In [88]:
fire_ev["borobox"]= np.select( [fire_ev["alarm_box_borough"] == "QUEENS",\
                               fire_ev["alarm_box_borough"] == "MANHATTAN",\
                               fire_ev["alarm_box_borough"] == "BRONX",\
                               fire_ev["alarm_box_borough"] == "BROOKLYN",\
                               fire_ev["alarm_box_borough"] == "STATEN ISLAND"], ["Q" + fire_ev["alarm_box_number_char"],\
                                                                         "M" + fire_ev["alarm_box_number_char"],\
                                                                         "X" + fire_ev["alarm_box_number_char"],\
                                                                         "B" + fire_ev["alarm_box_number_char"],\
                                                                         "R" + fire_ev["alarm_box_number_char"]])
fire_ev

,starfire_incident_id,incident_datetime,alarm_box_borough,alarm_box_number,alarm_box_location,incident_borough,zipcode,policeprecinct,citycouncildistrict,communitydistrict,...,ladders_assigned_quantity,other_units_assigned_quantity,incident_date,incident_month,incident_day,incident_year,incident_md,alarm_box_number_char,boro_box,borobox
0,5.001924e+14,2005-01-01T00:07:32.000,QUEENS,9237,N/SVC RD H. HARDING EXPY & 99 ST,QUEENS,11368.0,110.0,21.0,404.0,...,2,2,2005-01-01,1,1,2005,1-1,9237,QN9237,Q9237
1,5.001149e+14,2005-01-01T00:14:40.000,MANHATTAN,1493,BWAY & W125 ST\M.L.KING JR BLVD,MANHATTAN,10027.0,26.0,7.0,109.0,...,2,1,2005-01-01,1,1,2005,1-1,1493,MN1493,M1493
2,5.001088e+14,2005-01-01T00:16:29.000,MANHATTAN,878,2 AVE & 53 ST,MANHATTAN,10022.0,17.0,4.0,106.0,...,4,6,2005-01-01,1,1,2005,1-1,0878,MN0878,M0878
3,5.001088e+14,2005-01-01T00:16:29.000,MANHATTAN,878,2 AVE & 53 ST,MANHATTAN,10022.0,17.0,4.0,106.0,...,4,6,2005-01-01,1,1,2005,1-1,0878,MN0878,M0878
4,5.001065e+14,2005-01-01T00:24:58.000,BROOKLYN,653,LAFAYETTE & CLASSON AVES,BROOKLYN,11238.0,79.0,35.0,303.0,...,2,1,2005-01-01,1,1,2005,1-1,0653,BK0653,B0653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456371,2.112560e+15,2021-05-05T21:28:00.000,QUEENS,6117,CORBETT RD & 221 ST,QUEENS,11361.0,111.0,19.0,411.0,...,1,0,2021-05-05,5,5,2021,5-5,6117,QN6117,Q6117
456372,2.112510e+15,2021-05-05T21:31:00.000,MANHATTAN,1442,LEXINGTON AVE & 121 ST,MANHATTAN,10035.0,25.0,9.0,111.0,...,2,1,2021-05-05,5,5,2021,5-5,1442,MN1442,M1442
456373,2.112510e+15,2021-05-05T22:23:00.000,MANHATTAN,1325,AMSTERDAM AVE & 109 ST,MANHATTAN,10025.0,24.0,7.0,107.0,...,2,1,2021-05-05,5,5,2021,5-5,1325,MN1325,M1325
456374,2.112530e+15,2021-05-05T23:16:00.000,BRONX,3170,BRYANT AVE & BRONX PARK SO.,BRONX,10460.0,48.0,15.0,206.0,...,2,1,2021-05-05,5,5,2021,5-5,3170,BX3170,X3170


In [92]:
# create zipcode level file with counts
fire_ev_tracts= fire_ev.merge(alarm_box_t, on = "borobox", how = "outer")
fire_ev_tracts.to_csv(os.join(out, "fire_dispatch_tracts.csv"))

In [107]:
fire_tract_avgs= fire_ev_tracts.groupby("BoroCT2010")[["engines_assigned_quantity", "ladders_assigned_quantity", "highest_alarm_level",\
                                 "dispatch_response_seconds_qy"]].mean().reset_index()
fire_tract_counts= fire_ev_tracts["BoroCT2010"].value_counts().reset_index().rename({"index":"BoroCT2010", "BoroCT2010":"num_fire_ev"},\
                                                                                   axis = 1)
print(fire_tract_avgs)
print(fire_tract_counts)

fire_tract_sum= fire_tract_avgs.merge(fire_tract_counts, on = "BoroCT2010", how = "outer")
fire_tract_sum.head()

     BoroCT2010  engines_assigned_quantity  ladders_assigned_quantity  \
0       1000201                   3.212329                   2.089041   
1       1000202                   3.164295                   2.108668   
2       1000600                   3.014218                   2.052133   
3       1000700                   2.127660                   1.531915   
4       1000800                   3.292746                   2.163212   
...         ...                        ...                        ...   
2039    5030301                        NaN                        NaN   
2040    5030302                        NaN                        NaN   
2041    5031901                        NaN                        NaN   
2042    5031902                        NaN                        NaN   
2043    5032300                        NaN                        NaN   

      dispatch_response_seconds_qy  
0                        35.486301  
1                        40.923674  
2           

,BoroCT2010,engines_assigned_quantity,ladders_assigned_quantity,dispatch_response_seconds_qy,num_fire_ev
0,1000201,3.212329,2.089041,35.486301,148
1,1000202,3.164295,2.108668,40.923674,773
2,1000600,3.014218,2.052133,40.073460,422
3,1000700,2.127660,1.531915,44.164410,518
4,1000800,3.292746,2.163212,39.606218,387


In [106]:
fire_tract_sum.to_csv(os.join(out, "fire_tract_sum.csv"))

# Load data on vacate orders from HPD, due to fire
## From January 1st, 2017 through the present

* Order to repair/vacate orders: https://data.cityofnewyork.us/resource/tb8q-a3ar
* Fire Department building vacate list: https://data.cityofnewyork.us/resource/n5xc-7jfa
* Future work: combine old and new vacate orders? First dataset only has 93 records

In [6]:
url_vac_old = 'https://data.cityofnewyork.us/resource/n5xc-7jfa.csv?$limit=1000000'
vac_old = pd.read_csv(url_vac_old)
print("Size of data:", vac_old.shape)
# format date variable
vac_old["vac_date_orig"]= vac_old["vac_date"].copy()
vac_old["vac_date"]= pd.to_datetime(vac_old["vac_date"]).dt.date
vac_old["vac_year"]= pd.to_datetime(vac_old["vac_date"]).dt.year
# sort data
vac_old.sort_values("vac_year", ascending = True).head()

Size of data: (93, 19)


,vac_date,num,pf,location_1,typ,sf,bor,date_of_lif,area_vac,div,...,latitude,longitude,community_board,community_council,census_tract,bin,bbl,nta,vac_date_orig,vac_year
0,2008-05-19,1234,NaN,INTERVALE,AVE,NaN,BX,NaN,ENTIRE BLDG,6,...,40.828940,-73.894516,3.0,17.0,125.0,2010459.0,2.029740e+09,Morrisania-Melrose ...,5/19/2008,2008
1,2008-05-31,1556,NaN,PARKER,ST,NaN,BX,NaN,BASEMENT,6,...,40.838231,-73.850162,10.0,18.0,202.0,2041683.0,2.039720e+09,Westchester-Unionport ...,5/31/2008,2008
2,2009-03-17,1420,NaN,CROTONA,AVE,NaN,BX,NaN,ENTIRE BUILDING,6,...,40.834290,-73.898082,3.0,16.0,151.0,2009823.0,2.029370e+09,Morrisania-Melrose ...,3/17/2009,2009
3,2009-07-03,414,E,140,ST,NaN,BX,NaN,3rd floor and one room 2nd floor,6,...,40.810042,-73.921112,1.0,8.0,39.0,2000289.0,2.022840e+09,Mott Haven-Port Morris ...,7/3/2009,2009
13,2010-12-09,1443,NaN,TAYLOR,AVE,NaN,BX,NaN,BASEMENT FRONT,7,...,40.836275,-73.866172,9.0,18.0,218.0,2028224.0,2.038990e+09,West Farms-Bronx River ...,12/9/2010,2010


In [15]:
# load ALL vacate orders
url_vac_all = 'https://data.cityofnewyork.us/resource/tb8q-a3ar.csv?$limit=1000000'
vac_all = pd.read_csv(url_vac_all)
print(vac_all.shape)

# if false, then we might not have downloaded all data
print(vac_all.shape[0])
assert vac_all.shape[0] < 1000000
vac_all.to_csv(os.join(out, "all_vacate.csv"))

(5782, 20)
5782


In [16]:
# load vacate orders
url_vac = 'https://data.cityofnewyork.us/resource/tb8q-a3ar.csv?$limit=1000000&$where=primary_vacate_reason="Fire%20Damage"'
vac = pd.read_csv(url_vac)
print(vac.shape)

# if false, then we might not have downloaded all data
print(vac.shape[0])
assert vac.shape[0] < 1000000

(2904, 20)
2904


In [17]:
# create date, month, and year variables
vac["vacate_effective_date2"]= pd.to_datetime(vac["vacate_effective_date"]).dt.date
vac["vacate_effective_month"]= pd.to_datetime(vac["vacate_effective_date"]).dt.month
vac["vacate_effective_day"]= pd.to_datetime(vac["vacate_effective_date"]).dt.day
vac["vacate_effective_year"]= pd.to_datetime(vac["vacate_effective_date"]).dt.year

vac["vacate_effective_md"]= vac[["vacate_effective_month", "vacate_effective_day"]].astype(str).apply('-'.join, 1)

In [9]:
# number of fires per year
vac["vacate_effective_year"].value_counts().sort_index()

2012    283
2013    299
2014    250
2015    253
2016    241
2017    311
2018    312
2019    296
2020    274
2021    314
2022     71
Name: vacate_effective_year, dtype: int64

In [41]:
vac_sub= vac["bbl"].value_counts().reset_index().rename({"index":"bbl", "bbl":"num_vac_orders"}, axis = 1)
vac_sub

,bbl,num_vac_orders
0,2.028820e+09,15
1,3.013020e+09,7
2,2.045060e+09,4
3,2.026230e+09,4
4,2.036000e+09,4
...,...,...
2634,2.027520e+09,1
2635,2.033080e+09,1
2636,4.155160e+09,1
2637,2.026140e+09,1


In [37]:
## save file in output folder
vac.to_csv(os.join(out, "fire_vacate.csv"))
vac_sub.to_csv(os.join(out, "fire_vacate_bbl.csv"))

## Load PLUTO

In [21]:
## load pluto
pluto= gpd.read_file(os.join(inp, "nyc_mappluto_21v4_shp/MapPLUTO.shp"))
usecols = ["borough", "bbl", "cd", "ct2010", "zipcode", "address", "bldgclass", "landuse", "ownertype", "ownername",
           "lotarea", "bldgarea", "numbldgs", "numfloors", "unitsres", "unitstotal", "assessland", "assesstot",\
           "exempttot", "yearbuilt", "yearalter1", "yearalter2", "sanborn"]
pluto_df= pd.read_csv(os.join(inp, "nyc_pluto_21v4_csv/pluto_21v4.csv"),
                      usecols = usecols)
pluto_df.head()

,borough,cd,ct2010,zipcode,address,bldgclass,landuse,ownertype,ownername,lotarea,...,unitsres,unitstotal,assessland,assesstot,exempttot,yearbuilt,yearalter1,yearalter2,bbl,sanborn
0,MN,102.0,61.0,10003.0,42 EAST 12 STREET,R1,2.0,NaN,UNAVAILABLE OWNER,2643.0,...,6.0,6.0,292501.0,6494851.0,5800.0,1900.0,0.0,0.0,1.005638e+09,103 025
1,BK,315.0,392.0,11223.0,2105 OCEAN PARKWAY,A9,1.0,NaN,OCEAN PARKWAY BH 26 LLC,3445.0,...,1.0,1.0,18000.0,178800.0,0.0,2006.0,0.0,0.0,3.071330e+09,314 040
2,BK,315.0,388.0,11223.0,2287 EAST 1 STREET,B3,1.0,NaN,2287E1 LLC,1500.0,...,2.0,2.0,6360.0,59580.0,0.0,1930.0,0.0,0.0,3.071530e+09,314 047
3,QN,408.0,450.0,11432.0,161-26 GRAND CENTRAL PKWY,A5,1.0,NaN,CARLOS RODRIGUEZ,2925.0,...,1.0,1.0,15960.0,39060.0,1440.0,1935.0,0.0,0.0,4.068650e+09,406 042
4,QN,413.0,1617.0,11426.0,244-14 91 AVENUE,A9,1.0,NaN,BASHIR IRFAN,6086.0,...,1.0,1.0,26280.0,44220.0,1440.0,1935.0,2008.0,0.0,4.086580e+09,422 037


In [45]:
# left join from pluto to capture all properties
# use df for space constraints
vac_pluto_df= pluto_df.merge(vac, how = "left", on = "bbl", indicator = True)
print(vac_pluto_df["_merge"].value_counts())

# create indicator of fire vacancy based on indicator
vac_pluto_df["vacate_ind"]= np.where(
            vac_pluto_df["_merge"]=="both", 1, 0)

print("\nTab of outcome var (vacate ind)")
print(vac_pluto_df[["vacate_ind", "_merge"]].value_counts())
vac_pluto_df.drop("_merge", axis = 1, inplace = True)

left_only     856397
both            2889
right_only         0
Name: _merge, dtype: int64

Tab of outcome var (vacate ind)
vacate_ind  _merge   
0           left_only    856397
1           both           2889
dtype: int64


In [46]:
# save output to intermediate file
# create pluto and vacate orders
vac_pluto_df.to_csv(os.join(out, "vac_pluto.csv"))

In [52]:
## create same file at zipcode level for dispatch data
meancols = ["lotarea", "bldgarea", "numbldgs", "numfloors", "unitsres", "unitstotal", "assessland", "assesstot",\
           "exempttot", "yearbuilt"]
pluto_df_zip= pluto_df.groupby("zipcode")[meancols].mean().merge(
                        pluto_df.groupby("zipcode")[["bldgclass", "landuse"]].agg(pd.Series.mode), on = "zipcode", how = "outer")
pluto_df_zip

,lotarea,bldgarea,numbldgs,numfloors,unitsres,unitstotal,assessland,assesstot,exempttot,yearbuilt,bldgclass,landuse
zipcode,,,,,,,,,,,,
10001.0,1.377831e+04,104392.315361,1.048830,9.062462,22.253306,34.519837,2.642475e+06,1.205807e+07,4.756699e+06,1779.284553,O6,5.0
10002.0,1.091257e+04,25363.871287,1.094068,5.081336,21.099010,24.761211,5.827673e+05,2.453408e+06,1.162279e+06,1755.149097,C7,4.0
10003.0,5.200328e+03,29568.027508,1.091694,5.848840,18.045307,20.670982,8.438337e+05,4.074944e+06,6.899876e+05,1870.140237,C7,4.0
10004.0,1.510340e+05,258934.991304,2.391304,14.452174,34.269565,50.547826,9.428934e+06,2.545286e+07,9.323778e+06,1800.600000,O4,5.0
10005.0,2.217579e+04,387247.596774,1.048387,23.568548,95.161290,117.225806,1.051464e+07,3.703690e+07,1.344642e+06,1900.693548,O4,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
11693.0,1.793147e+04,3224.394094,1.101324,1.621146,2.400713,2.585031,1.042085e+05,1.688853e+05,1.269630e+05,1596.960794,A1,1.0
11694.0,1.307254e+04,3619.022181,1.551844,2.007728,2.489311,2.637092,7.975477e+04,1.749641e+05,8.745940e+04,1858.569749,A1,1.0
11695.0,1.651950e+06,3000.000000,3.250000,0.250000,0.000000,0.000000,3.161925e+07,3.237964e+07,3.237964e+07,482.250000,Q0,9.0


,bldgclass,landuse
zipcode,,
10001.0,O6,5.0
10002.0,C7,4.0
10003.0,C7,4.0
10004.0,O4,5.0
10005.0,O4,5.0
...,...,...
11693.0,A1,1.0
11694.0,A1,1.0
11695.0,Q0,9.0


## Load data on electricity and water usage
* [2020 data](https://data.cityofnewyork.us/resource/usc3-8zwd.csv)
* [2019 data](https://data.cityofnewyork.us/resource/wcm8-aq5w.csv)
* [2018 data](https://data.cityofnewyork.us/resource/4tys-3tzj.csv)
* [2017 data](https://data.cityofnewyork.us/resource/4t62-jm4m.csv)

In [46]:
# store urls
url_2020= "https://data.cityofnewyork.us/resource/usc3-8zwd.csv?$limit=100000"
url_2019= "https://data.cityofnewyork.us/resource/wcm8-aq5w.csv?$limit=100000"
url_2018= "https://data.cityofnewyork.us/resource/4tys-3tzj.csv?$limit=100000"
url_2017= "https://data.cityofnewyork.us/resource/4t62-jm4m.csv?$limit=100000"

# load and append data from each year
urls= [url_2017, url_2018, url_2019, url_2020]
cols= [["property_id", "year_ending",\
                                      "property_name", "nyc_borough_block_and_lot",\
                                      "occupancy", "year_built", "number_of_buildings",\
                                       "water_use_all_water_sources", "multifamily_housing_number",\
                                        "multifamily_housing_total"],\
      ["property_id", "year_ending",\
                                      "property_name", "nyc_borough_block_and_lot",\
                                      "occupancy", "year_built", "number_of_buildings",\
                                       "water_use_all_water_sources", "multifamily_housing_number",\
                                        "multifamily_housing_total"],\
      ["property_id", "year_ending",\
                                      "property_name", "nyc_borough_block_and_lot",\
                                      "occupancy", "year_built", "number_of_buildings",\
                                       "water_use_all_water_sources", "multifamily_housing_number",\
                                        "multifamily_housing_total"],\
      ["property_id", "year_ending",\
                                      "property_name", "nyc_borough_block_and_lot_bbl",\
                                      "occupancy", "year_built", "number_of_buildings",\
                                       "water_use_all_water_sources_kgal", "multifamily_housing_number_of_bedrooms",\
                                        "multifamily_housing_total_number_of_residential_living_units"]]
# read in all data files using lc, combine across the rows
dfs= [pd.read_csv(urls[i], usecols = cols[i]) for i in range(0, 4)]
dfs

[       property_id              year_ending  \
 0          4593574  2017-12-31T00:00:00.000   
 1          6224375  2017-12-31T00:00:00.000   
 2          2967701  2017-12-31T00:00:00.000   
 3          4898531  2017-12-31T00:00:00.000   
 4          2917939  2017-12-31T00:00:00.000   
 ...            ...                      ...   
 34681      4047752  2017-12-31T00:00:00.000   
 34682      6682473  2017-12-31T00:00:00.000   
 34683      6682474  2017-12-31T00:00:00.000   
 34684      6682477  2017-12-31T00:00:00.000   
 34685      2819133  2017-12-31T00:00:00.000   
 
                                 property_name  nyc_borough_block_and_lot  \
 0                       The Argonaut Building                 1010287502   
 1                             Operative Cakes  2-00560-0062;2-05560-0062   
 2              Cathedral Preparatory Seminary               4-01872-0007   
 3                             The Nomad Hotel               1-00829-0050   
 4                      10 West 27 St

In [47]:
# make names of columns uniform
dfs[0].columns = cols[0]
dfs[1].columns = cols[0]
dfs[2].columns = cols[0]
dfs[3].columns = cols[0]

# concatenate along rows (i.e. append data from each year)
df_ew= pd.concat(dfs, axis = 0)
df_ew

,property_id,year_ending,property_name,nyc_borough_block_and_lot,occupancy,year_built,number_of_buildings,water_use_all_water_sources,multifamily_housing_number,multifamily_housing_total
0,4593574,2017-12-31T00:00:00.000,The Argonaut Building,1010287502,1909,1,95,NaN,NaN,3635.5
1,6224375,2017-12-31T00:00:00.000,Operative Cakes,2-00560-0062;2-05560-0062,1973,2,100,NaN,NaN,116.0
2,2967701,2017-12-31T00:00:00.000,Cathedral Preparatory Seminary,4-01872-0007,1963,1,100,NaN,NaN,102.9
3,4898531,2017-12-31T00:00:00.000,The Nomad Hotel,1-00829-0050,1999,1,85,NaN,NaN,10762.6
4,2917939,2017-12-31T00:00:00.000,10 West 27 Street Corp,1-00828-0053,1994,1,100,NaN,NaN,790.1
...,...,...,...,...,...,...,...,...,...,...
28062,17286197,ACF - 129th St Residence - 109 West 129th St: ...,2020-12-31T00:00:00.000,1-01914-0026,1920,1,100,Not Available,Not Available,Not Available
28063,17298485,110-31 Merrick Boulevard,2020-12-31T00:00:00.000,4102700059,1997,1,100,Not Available,Not Available,Not Available
28064,17298486,110-31A Merrick Boulevard,2020-12-31T00:00:00.000,4102700047,1997,1,100,Not Available,Not Available,Not Available
28065,17321529,402 EAST 78 STREET,2020-12-31T00:00:00.000,1014720046,1910,1,100,52,39,1788.6


In [48]:
# text formatting of bbl
df_ew.loc[:, "bbl"]= df_ew.iloc[:, 3].str.replace("[A-z]{1}[0-9]{3}-", "", regex = True)
df_ew.loc[:, "bbl"]= df_ew.loc[:, "bbl"].str.replace("-", "").str.replace(" ", ";")\
                        .str.replace("(?<=[0-9]{10})/", ";", regex = True)\
                        .str.replace("/", "")\
                        .str.replace(",", ";")\
                        .str.replace(":", ";")\
                        .str.replace("and", ";")\
                        .str.replace("&", ";")\
                        .str.replace("NotAvailable", "")\
                        .str.replace("multiple", "")
df_ew.loc[:, "bbl"]

0                   1010287502
1        2005600062;2055600062
2                   4018720007
3                   1008290050
4                   1008280053
                 ...          
28062               1019140026
28063               4102700059
28064               4102700047
28065               1014720046
28066               4102700057
Name: bbl, Length: 117082, dtype: object

In [49]:
# need to split bbl into several columns
max_num= int(df_ew["bbl"].str.count(";").max())
df_ew[["bbl" + str(i) for i in range(0, (max_num + 1))]]= df_ew["bbl"].str.split(pat= ";", n=-1, expand = True)
df_ew[["bbl" + str(i) for i in range(0, (max_num + 1))]].head()

,bbl0,bbl1,bbl2,bbl3,bbl4,bbl5,bbl6,bbl7,bbl8,bbl9,...,bbl32,bbl33,bbl34,bbl35,bbl36,bbl37,bbl38,bbl39,bbl40,bbl41
0,1010287502,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,2005600062,2055600062,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,4018720007,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,1008290050,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,1008280053,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [50]:
# reshape and reformat bbl column
# pivot data long using melt
cols = [x for x in df_ew.columns if x not in ["bbl" + str(i) for i in range(0, max_num)] + ["bbl"]]
cols

# melt pivots data long
df_ewlong= pd.melt(df_ew.drop("bbl", axis = 1), id_vars=cols,var_name='bbl_num', value_name='bbl')
# recode comparison
print(df_ewlong[["bbl", "nyc_borough_block_and_lot"]].dropna().loc[(df_ewlong["bbl"].dropna()).map(len) > 10, :])

df_ewlong= df_ewlong[(df_ewlong["bbl"].notna()) | (df_ewlong["bbl"].isna() & \
                                                         df_ewlong["bbl_num"] == "bbl0")].drop_duplicates()

                  bbl                          nyc_borough_block_and_lot
61      0004033410037                                    0004-03341-0037
319       1005800065.                                      1-00580-0065.
2061      10193100006                                      1-01931-00006
4018      1005800065.                                      1-00580-0065.
4290      20028110024                                        20028110024
...               ...                                                ...
482954    05006200200  05-00620-0001; 05-00620-0100; 05-00620-0200; 0...
516925    30605000019  3-06050-0051; 3-06050-0059; 3-06050-00019; 3-0...
541896    30605000019    3060500051; 3060500059; 30605000019; 3060500015
569611    30605000019    3060500051; 3060500059; 30605000019; 3060500015
717118    05006200300  05-00620-0001; 05-00620-0100; 05-00620-0200; 0...

[69 rows x 2 columns]


In [51]:
# drop these values, not useful
df_ewlong= df_ewlong.drop(df_ewlong.loc[(df_ewlong["bbl"].str.len() != 10) | \
                (df_ewlong["bbl"] == "Code9Code9") | \
                (df_ewlong["bbl"] == "Not Available") | \
                (df_ewlong["bbl"] == "XXXXXXXXXX"), :].index, axis = 0)

In [52]:
# clean total housing units data
df_ewlong.loc[(df_ewlong["multifamily_housing_total"] == "0") | \
              (df_ewlong["multifamily_housing_total"] == "Not Available"), "multifamily_housing_total"]= ""
df_ewlong["multifamily_housing_total_orig"]= df_ewlong["multifamily_housing_total"].copy()
df_ewlong["multifamily_housing_total"]= pd.to_numeric(df_ewlong["multifamily_housing_total"])

df_ewlong[["multifamily_housing_total", "multifamily_housing_total_orig"]].sample(frac=0.05).value_counts()

multifamily_housing_total  multifamily_housing_total_orig
0.0                        0.0                               8
2538.9                     2538.9                            3
6980.3                     6980.3                            3
1785.1                     1785.1                            2
1473.7                     1473.7                            2
                                                            ..
2358.3                     2358.3                            1
2359.4                     2359.4                            1
2360.1                     2360.1                            1
2364.4                     2364.4                            1
4095889.9                  4095889.9                         1
Length: 3258, dtype: int64

In [53]:
# calculate water usage per unit (can also be calculated for comparison using resunits from pluto)
df_ewlong["water_num"]= pd.to_numeric(df_ewlong["water_use_all_water_sources"].str.replace("Not Available", ""))
# fill missing values with 0s
df_ewlong["pwater_num"]= df_ewlong["water_num"].div(df_ewlong["multifamily_housing_total"].astype(float), \
                                                       fill_value = 0).fillna(0)

In [54]:
# merge to pluto and main df
print(df_ewlong[["bbl","pwater_num", "water_num", "multifamily_housing_total"]].head())
# need to convert bbl to string
vac_census["bbl"]= vac_census["bbl"].astype(str).str.replace(".0", "", regex = False)
assert (vac_census["bbl"].str.len() == 10).all()
assert (df_ewlong["bbl"].str.len() == 10).all()

vac_ew= pd.merge(vac_census, df_ewlong, how = "left", on = "bbl", indicator = True)

          bbl  pwater_num  water_num  multifamily_housing_total
0  1010287502         0.0        NaN                     3635.5
1  2005600062         0.0        NaN                      116.0
2  4018720007         0.0        NaN                      102.9
3  1008290050         0.0        NaN                    10762.6
4  1008280053         0.0        NaN                      790.1


In [55]:
print(vac_ew.shape)
# check size of unmerged buildings (small buildings under 10,000 sq. ft. for gov't buildings
# and 25,000 shouldn't be merging)
print(vac_ew.loc[vac_ew["property_id"].notna(), :].shape)
print(vac_ew.loc[vac_ew["property_id"].isna(), :].shape)
print(vac_ew.loc[(vac_ew["property_id"].isna()) & \
                       (vac_ew["bldgarea"] < 25000), "bldgarea"].shape)

print("\nMerge rate:")
print(vac_ew["_merge"].value_counts())
# assert there are no values unmerged on rhs
assert vac_ew.loc[vac_ew["_merge"]=="right_only"].shape[0]==0
vac_ew.drop("_merge", axis = 1, inplace = True)

(939431, 93)
(107179, 93)
(832252, 93)
(828287,)

Merge rate:
left_only     832252
both          107179
right_only         0
Name: _merge, dtype: int64


## Load HPD Housing Code Violations
* Data saved on the [NYC Open Data Portal](https://data.cityofnewyork.us/resource/wvxf-dwi5.csv)
* Load Class C Violations (Class C are most severely hazardous)
* All violation filings since January 1st, 2017

In [57]:
import psycopg2 as psy
# load furman db
connection = psy.connect(
    host="fcdata.c7h93yqbqnvu.us-east-1.rds.amazonaws.com",
    port=5432,
    user = "furmandata",
    password = config.furman_pwd,
    database = "fcdata")
cursor = connection.cursor()

In [58]:
viol_summary= pd.read_sql("""SELECT violation_class, bbl, Count(*) FROM hpd_violations
                            WHERE inspection_date >= '2017-01-01'
                            GROUP BY violation_class, bbl;""", con=connection)
viol_summary= viol_summary.drop(viol_summary[(viol_summary["bbl"]=='0         ') | \
                                             (viol_summary["bbl"].isna())].index, axis = 0)
assert (viol_summary["bbl"].str.len() == 10).all()
viol_summary.head()

,violation_class,bbl,count
1,A,1000070038,1
2,A,1000077501,15
3,A,1000077502,1
4,A,1000080039,1
5,A,1000100033,1


In [ ]:
# convert bbl to string
viol_summary["bbl"]= viol_summary["bbl"].astype(str).str.replace(".0", "", regex = False)
viol_summary= viol_summary.rename({"count":"num_viol"}, axis = 1)

In [ ]:
# pivot data wider
viol_wide= viol_summary.pivot(index='bbl', columns='violation_class', values = 'num_viol').fillna(0).reset_index()
viol_wide.head()

In [ ]:
# check length of bbl prior to merging
assert (vac_ew["bbl"].str.len() == 10).all()
assert (viol_wide["bbl"].str.len() == 10).all()
vac_viol= pd.merge(vac_ew, viol_wide, how = "left", on = "bbl", indicator = True)
print("Merge rate:")
print(vac_viol["_merge"].value_counts())
vac_viol.drop("_merge", axis = 1, inplace = True)

In [ ]:
print("Average number of class C violations:", vac_viol["C"].mean())
print("Average number of class C violations (among buildings w fire vacate orders):", \
      vac_viol.loc[vac_viol["vacate_ind"] == 1, "C"].mean())

# replace na values with 0
viol_cols= ["A", "B", "C", "I"]
vac_viol.loc[:, viol_cols]= vac_viol.loc[:, viol_cols].fillna(0)
# rename cols
vac_viol.rename({i:j for i,j in zip(viol_cols, ["numviol_" + x for x in viol_cols])}, axis = 1, inplace = True)
vac_viol.loc[:, ["numviol_" + x for x in ["A", "B", "C", "I"]]]

## Save cleaned analytic file
* Create version of data file with only selected variables that can be used for the analysis

In [ ]:
vac_viol["pwater_num_na"]= np.where(
    vac_viol["pwater_num"].isna() | np.isinf(vac_viol["pwater_num"]) | vac_viol["pwater_num"] == 0, 1, 0)
vac_viol["pwater_num"]= np.where(
    vac_viol["pwater_num"].isna() | np.isinf(vac_viol["pwater_num"]), 0, vac_viol["pwater_num"])
print(vac_viol[["pwater_num_na", "pwater_num"]].value_counts())

analysis_vars= ["bbl", "unitsres", "yearbuilt", "medhhinc", "assessland",\
                    "crowding_ind", "pwater_num", "pwater_num_na", "exempttot", "numviol_C", "vacate_ind"]
print(vac_viol.columns)
df_analysis= vac_viol.loc[:, analysis_vars]

In [ ]:
# export
df_analysis.to_csv(os.join(root, "data/analysis_file.csv"))